In [35]:
import time
import cv2
import cv2.aruco as aruco
import numpy as np
from realsense_depth import *
# import the necessary packages
from threading import Thread
from utlities import *
from control_class import *
from scipy import fftpack

## IMAGE RESOLUTION
WIDTH = 1280
HEIGHT = 720
timeRecord = []
xRecord = []
xFilteredRecord = []
yRecord = []
yFilteredRecord = []
zRecord = []
zFilteredRecord = []
angleRecord = []
angleFilteredRecord = []
xErrorRecord = []
yErrorRecord = []
zErrorRecord = []
angleErrorRecord = []
aileronRecord = []
elevatorRecord = []
throttleRecord = []
rudderRecord = []
def drone_pose(frame ,matrix_coefficients, distortion_coefficients,id_no):
    ## MARKER SIZE OF DRONE
    MARKER_SIZE = 0.056
    rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[id_no], MARKER_SIZE, matrix_coefficients,distortion_coefficients)
    return frame,tvec,rvec

# defining an empty custom dictionary 
arucoDict = cv2.aruco.custom_dictionary(0, 4, 1)
# adding empty bytesList array to fill with 5 markers 
arucoDict.bytesList = np.empty(shape = (5, 2, 4), dtype = np.uint8)

# adding new markers
mybits = np.array([[0,1,0,0],[1,1,0,0],[1,0,1,0],[1,1,0,1]], dtype = np.uint8)
arucoDict.bytesList[0] = cv2.aruco.Dictionary_getByteListFromBits(mybits)
mybits = np.array([[1,1,1,1],[1,0,0,1],[1,0,0,1],[0,0,0,1],], dtype = np.uint8)
arucoDict.bytesList[1] = cv2.aruco.Dictionary_getByteListFromBits(mybits)
mybits = np.array([[0,0,0,1],[0,0,0,1],[1,0,1,0],[0,1,1,1]], dtype = np.uint8)
arucoDict.bytesList[2] = cv2.aruco.Dictionary_getByteListFromBits(mybits)
mybits = np.array([[0,0,0,0],[1,1,1,0],[1,0,1,1],[0,1,1,1],], dtype = np.uint8)
arucoDict.bytesList[3] = cv2.aruco.Dictionary_getByteListFromBits(mybits)
mybits = np.array([[0,0,1,0],[1,0,1,0],[0,0,0,0],[1,1,1,1]], dtype = np.uint8)
arucoDict.bytesList[4] = cv2.aruco.Dictionary_getByteListFromBits(mybits)

arucoParams = aruco.DetectorParameters_create()

calibration_matrix_path = "calibration_matrix.npy"
distortion_coefficients_path = "distortion_coefficients.npy"
    
k = np.load(calibration_matrix_path)
d = np.load(distortion_coefficients_path)

# initialize the video stream and allow the camera sensor to warm up
print("[INFO] starting video stream...")

time.sleep(2.0)

# Initialize the video stream
dc = DepthCamera(width=WIDTH, height=HEIGHT)
first_time = True

num_not_detected = 0
eps = 0.1
desired_depth = 2


# PID PARAMETERS

command = Command("192.168.4.1")


## MEAN - HOVERING VALUES
# mean_roll =  1511
# mean_pitch = 1482
# mean_throttle = 1475
Kp_T = 15
Ki_T = 2
Kd_T = 1

I_P = 0
I_T = 0
I_R =0
I_Y = 0

e_P =0
e_R = 0
e_T = 0
e_Y = 0

Kp_R = 20
Ki_R = 2
Kd_R = 0.25

Kp_P = 20
Ki_P = 2
Kd_P = 0.25

Kp_Y = 0

mean_roll =  1510
mean_pitch = 1505
mean_throttle = 1465
mean_yaw = 1500
# mean_roll =  1500
# mean_pitch = 1500
# mean_throttle = 1500
# command.roll = 1500
# command.pitch = 1487
# command.throttle = 1500
slope = 0
for i in range(10):
    command.disarm()
    time.sleep(0.1)
# command.calib()
num_samples = 0
time.sleep(0.3)
for i in range(10):
    command.disarm()
    time.sleep(0.1)
command.takeoff()
# time.sleep(0.2)
prev_time = time.time()
start = prev_time
print('[INFO] : Takeoff Completed')
try:
    while True:

        ret, color_frame = dc.get_frame()

        gray = cv2.cvtColor(color_frame,cv2.COLOR_BGR2GRAY)
        (corners, ids, rejected) = cv2.aruco.detectMarkers(gray,arucoDict, parameters=arucoParams)

        if len(corners)>0 and 0 in ids : 
                #print('Tracking drone')
                #print(corne)
                
                for i in range(0,len(ids)):
                    
                    if ( abs(corners[i][0][1][0] - corners[i][0][0][0]) > 100 or abs(corners[i][0][1][1] - corners[i][0][0][1]) > 100 ) :
                        print(' [ERROR] : Large Marker detected ...')
                    if ids[i] == 0:
                        color_frame,xc,yc = aruco_display(color_frame,corners[i])
                        frame,tvec,rvec = drone_pose(color_frame,k,d,i)
                        x = tvec[0][0][0]
                        y = tvec[0][0][1]

                        point1 = corners[i][0][0] + corners[i][0][3]
                        point2 = corners[i][0][1] + corners[i][0][2]
                        #slope =  (point1[1] - point2[1])/(point1[0] - point2[0])
                    num_not_detected = 0
                    num_samples+=1
        else :
            print('[INFO] : Drone not detected ...')
            if num_not_detected > 25:
                print('[INFO] : Drone out of range...')
                print('[INFO] : Landing ...')
                command.land()
                print('[INFO] : Quitting ...')
                break

            num_not_detected +=1
            command.send()
            #time.sleep(0.1)
            continue
        xRecord.append(x)
        yRecord.append(y)
        zRecord.append(depth)
        ## HOVERING 
        if first_time:
            desired_pos = np.array([x,y,desired_depth])
            first_time = False
        else:
            depth = dc.get_depth(xc,yc)
            curr_pos = np.array([x,y,depth])
        
            if(np.linalg.norm(desired_pos - curr_pos) < 0.2):
                print('[INFO] : Reached Position')
                
            print('desired pos : ',desired_pos)
            print('curr_pos :', curr_pos )

            curr_time = time.time()

            print('pitch')
            correction_x,I_P,e_P = PID(Kp_P,Ki_P,Kd_P,curr_pos[0],desired_pos[0],e_P,I_P,curr_time-prev_time)
            print('roll')
            correction_y,I_R,e_R = PID(Kp_R,Ki_R,Kd_R,curr_pos[1],desired_pos[1],e_R,I_R,curr_time-prev_time)
            print('throttle')
            correction_z,I_T,e_T = PID(Kp_T,Ki_T,Kd_T,curr_pos[2],desired_pos[2],e_T,I_T,curr_time-prev_time)
            print('angle')
            correction_a = Kp_Y*slope


            # command.pitch -= int(correction_x)
            # command.roll -= int(correction_y) 
            # command.throttle -=int(correction_z)

            command.pitch = mean_pitch - int(correction_x)
            command.roll = mean_roll - int(correction_y)
            command.throttle = mean_throttle - int(correction_z)
            command.yaw = mean_yaw - int(correction_a)

            # Clamping values between 900 and 2100
            command.throttle = max(900,min(command.throttle,2100))
            command.pitch = max(900,min(command.pitch,2100))
            command.roll = max(900,min(command.roll,2100))
            command.yaw = max(900,min(command.yaw,2100))
            command.send()

            print('Pitch : ',command.pitch)
            print('Roll : ',command.roll)
            print('Throttle : ',command.throttle)

            cv2.imshow("Frame", color_frame)

            
        key = cv2.waitKey(1) & 0xFF
        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

        ## Updating the current time for FPS Calculation
        curr_time = time.time()
        fps = 1/(curr_time-prev_time)
        prev_time = curr_time
        print('[INFO] : fps = ',fps)

        # Restricting FPS
        # time.sleep(0.01)

except KeyboardInterrupt:
    print('[INFO] : Keyboard Interrupt')
    command.boxarm()
    command.land()
# except:
#     print('[INFO] : Exception')
#     command.boxarm()
#     command.land()

dc.release()
cv2.destroyAllWindows()
end = time.time()


[INFO] starting video stream...
DepthCamera
[INFO] : Takeoff Completed
[INFO] : Drone not detected ...
[INFO] : Drone detected
[INFO] : fps =  4.014360299571699
[INFO] : Drone detected
desired pos :  [-0.13867151 -0.17507537  2.        ]
curr_pos : [-0.13505237 -0.17396604  2.93700004]
pitch
roll
throttle
angle
Pitch :  1505
Roll :  1510
Throttle :  1500
[INFO] : fps =  12.690975657725533
[INFO] : Drone detected
desired pos :  [-0.13867151 -0.17507537  2.        ]
curr_pos : [-0.1357276  -0.17659838  2.92800021]
pitch
roll
throttle
angle
Pitch :  1505
Roll :  1510
Throttle :  1478
[INFO] : fps =  14.885876123280475
[INFO] : Drone detected
desired pos :  [-0.13867151 -0.17507537  2.        ]
curr_pos : [-0.1294238  -0.17362455  2.92000008]
pitch
roll
throttle
angle
Pitch :  1505
Roll :  1510
Throttle :  1478
[INFO] : fps =  16.595265508959766
[INFO] : Drone detected
desired pos :  [-0.13867151 -0.17507537  2.        ]
curr_pos : [-0.12494703 -0.17769512  2.89500022]
pitch
roll
throttle


In [1]:
from numpy.fft import fft,fftfreq
import matplotlib.pyplot as plt
# time  = end - start
# fft_x = fftpack.fft(xRecord)
# fft_y = fftpack.fft(yRecord)
# fft_z = fftpack.fft(zRecord)
# #print(fft_)
# X_amp = 2 / ((time/num_samples) * np.abs(fft_x))
# Y_amp = 2/ ((time/num_samples) * np.abs(fft_y))
# Z_amp = 2/ ((time/num_samples) * np.abs(fft_z))
# sig_noise_freq = np.abs(fftpack.fftfreq(num_samples,time/num_samples))

# plt.plot(sig_noise_freq,X_amp)
sr = (end - start)/num_samples
X = fft(xRecord)
Y = fft(yRecord)
Z = fft(zRecord)
N = len(X)
n = np.arange(N)
T = N/sr
#freq = n/T
print(np.abs(X))
freq = fftfreq(N,sr)

plt.plot(freq[:N//4],np.abs(X)[:N//4])
plt.show()
plt.plot(freq[:N//4],np.abs(Y)[:N//4])
plt.show()
plt.plot(freq[:N//4],np.abs(Z)[:N//4])
plt.show()


NameError: name 'end' is not defined